<img width="20%" alt="EarthDaily Agro" src="https://earthdaily.com/hubfs/EDA_logo_horizontal_gradient-navy_1.svg" style="border-radius: 12px" />

# Client & Use Case Management

> Authenticate, list customers, inspect groups, and bootstrap a demo use case.

**Notebook map**
- 🔐 Setup & authenticate (imports, `.env`, API session)
- 🧭 Explore customers/groups (list tenants, select one, launch provisioning)
- 📦 Download artifacts + enrich them with crop history
- 📤 Persist an enriched baseline for later notebooks


## Before you start

- Keep credentials in `.env` so you can switch environments without editing the notebook.
- The helper functions (download, enrichment, map preview) live under `lib/` and are imported once during setup.
- Sections include mini tables describing what you can tweak (customer codes, geometry payload, crop-mask options, export paths).


In [ ]:
# --- bootstrap, environment, and session ------------------------------------------
import logging
import sys
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown


logging.basicConfig(level=logging.INFO)

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import pandas as pd
import requests
from lib import (
    ApiSession,
    download_parquet_to_df,
    filter_crop_history_columns,
    preview_geometries_map,
    publish_group_artifact,
    wait_for_job,
)
from lib.analytics import (
    AnalyticsRunReporter,
    BindingKey,
    CropMaskParams,
    apply_notebook_style,
    format_bindings_catalog_html,
    render_final_report,
    run_task_with_reporting,
)

apply_notebook_style()

# Authentication context (.env is shared with the other notebooks)
env_path = Path('..') / '.env'
if env_path.exists():
    load_dotenv(env_path)

session = ApiSession.from_env()
session


## Quick navigation
- [Before you start](#Before-you-start)
- [Logging controls](#Logging-controls)
- [List customers](#1.-List-customers)
- [Select a customer code](#2.-Select-a-customer-code)
- [Initialize a new group](#3.-Initialize-a-new-group-(use-case))
- [List use cases](#4.-List-use-cases-for-the-selected-customer)
- [Download artifact](#5.-Download-the-initial-artifact-(presigned-URLs))
- [Preview spatial units](#6.-Preview-spatial-units-on-a-map)
- [Crop mask enrichment](#7.-Enrich-with-crop-mask-information)
- [Publish](#8.-Publish-the-enriched-baseline)


### Logging controls

This cell configures logging for this notebook. All analytics loggers start at INFO; adjust individual tasks if you need more or less verbosity.


In [ ]:
# Notebook logging: set analytics loggers to INFO by default.
logging.getLogger().setLevel(logging.INFO)
logging.getLogger("lib.analytics").setLevel(logging.INFO)
logging.getLogger("analytics.crop_mask").setLevel(logging.INFO)

# Example: raise or lower verbosity per task (e.g., DEBUG for troubleshooting)
# logging.getLogger("analytics.crop_mask").setLevel(logging.DEBUG)


## 1. List customers

Use the management API to enumerate every tenant accessible with the current credentials. Typical fields: `name`, `code`, plus any metadata returned by your environment.

---


In [ ]:
customers = session.get('/v1/customers')
customers

## 2. Select a customer code

Pick the tenant you want to highlight. The snippet defaults to the first row for convenience—replace it with a widget or direct assignment when demoing specific accounts.

---


In [ ]:
customer_record = customers[0]  # Replace with a selector for interactive demos if needed
customer_code = customer_record['code']

display(Markdown(f"**Selected customer:** {customer_record['name']} (`{customer_code}`)"))
customer_record

## 3. Initialize a new group (use case)

Seed a polygon and launch the async provisioning job. Every field below can be edited quickly when walking through workshops.

| Parameter | Description |
| --- | --- |
| `customer_code` | Tenant that will own the group |
| `label` | Display name in portals |
| `no_overlap` | Optional guardrail to prevent geometry overlap |
| `require_*` flags | Force readiness for specific downstream pipelines |

Use the same WKT you ingested in Notebook 1 to keep the story coherent.

---


In [ ]:
geometry_wkt = [
    "POLYGON ((-94.65769669000616 44.25048284611244, -94.65800279739844 44.24607247866539, -94.6589087256991 44.24170490686098, -94.66040562153817 44.237422178804046, -94.66247895039672 44.23326551684501, -94.6651086411286 44.22927492146055, -94.66826928323373 44.22548878719971, -94.67193037483094 44.221943534348874, -94.67605661881908 44.21867325980829, -94.68060826428255 44.21570941047953, -94.68554148979818 44.21308048224219, -94.69080882493432 44.21081174734897, -94.69635960590416 44.208925012794204, -94.70214046104353 44.207438411916186, -94.7080958215316 44.20636623117867, -94.71416845256185 44.20571877374519, -94.7203 44.205502261116536, -94.72643154743815 44.20571877374519, -94.7325041784684 44.20636623117867, -94.73845953895648 44.207438411916186, -94.74424039409584 44.208925012794204, -94.74979117506568 44.21081174734897, -94.75505851020182 44.21308048224219, -94.75999173571745 44.21570941047953, -94.76454338118093 44.21867325980829, -94.76866962516907 44.221943534348874, -94.77233071676628 44.22548878719971, -94.7754913588714 44.22927492146055, -94.7781210496033 44.23326551684501, -94.78019437846183 44.237422178804046, -94.7816912743009 44.24170490686099, -94.78259720260156 44.24607247866539, -94.78290330999384 44.25048284611244, -94.78260651446156 44.25489353974701, -94.78170954017224 44.259262077578775, -94.78022089639904 44.26354637437918, -94.77815480053916 44.267705147512046, -94.7755310457788 44.271698315369655, -94.77237481450182 44.27548738454453, -94.76871643908322 44.279035821963326, -94.76459111224214 44.28230940834478, -94.76003854964782 44.28527656951522, -94.75510260796733 44.287908682322715, -94.74983086201223 44.29018035213324, -94.7442741450741 44.29206965916505, -94.73848605693287 44.29355837122074, -94.73252244436974 44.29463212070502, -94.72644085931437 44.29528054416805, -94.7203 44.29549738298498, -94.71415914068564 44.29528054416805, -94.70807755563027 44.29463212070502, -94.70211394306713 44.29355837122074, -94.6963258549259 44.29206965916506, -94.69076913798777 44.29018035213324, -94.68549739203267 44.28790868232273, -94.68056145035219 44.285276569515226, -94.67600888775786 44.28230940834478, -94.67188356091678 44.279035821963326, -94.66822518549819 44.27548738454453, -94.66506895422121 44.271698315369655, -94.66244519946083 44.267705147512046, -94.66037910360096 44.26354637437918, -94.65889045982776 44.25926207757878, -94.65799348553844 44.25489353974701, -94.65769669000616 44.25048284611244))"
]

group_params = {
    'customer_code': customer_code,
    'label': 'Demo Test',
    'no_overlap': True,
    'require_lri_ready': True,
    'require_weather_ready': True,
}
response = session.post('/v1/groups', params=group_params, json={'geometry_wkt': geometry_wkt})

# set the group id to be used in next steps
group_id = response['group']['group_id']

job_id = response['job']['job_id']

job_result = wait_for_job(session, job_id, lambda sess, identifier: sess.get(f'/v1/jobs/{identifier}'))

display(Markdown("------"))
display(Markdown(f"## ✅ {group_params['label']} — Group Id"))
display(Markdown(f"### `{group_id}`"))
display(Markdown("------"))

display(Markdown(job_result["summary"]))

## 4. List use cases for the selected customer

Immediately after provisioning, refresh the catalog of groups to confirm the new use case appears (and to show what already exists). Override `group_id` above to inspect historical groups instead of the newly created one.

---


In [ ]:
# group_id = "4e607157-66c8-4adf-9439-8289b79df20d"  # you can override the group_id value

# Tip: set group_id above to the ID shown in the previous cell if you want to reuse it
groups = session.get('/v1/groups')

# Display all groups
groups_df = pd.DataFrame(groups)
display(groups_df)

## 5. Download the initial artifact (presigned URLs)

Use presigned URLs to pull down the parquet + manifest.

| Knob | Usage |
| --- | --- |
| `group_id` | Default to the freshly created group or override it manually |
| `artifact_type` | `parquet` vs `manifest` |
| `version` | Start with `latest`, override if you need an older version |

---


In [ ]:
# Reuse the group we just provisioned; override `group_id` above if you want to inspect a different use case.
artifacts = session.get(f'/v1/groups/{group_id}/artifacts')
if not artifacts:
    raise RuntimeError('No artifacts available for this group yet.')
latest_version = artifacts[0]['version']

parquet_url = session.get(f'/v1/groups/{group_id}/presign', params={'artifact_type': 'parquet', 'version': latest_version})['url']
manifest_url = session.get(f'/v1/groups/{group_id}/presign', params={'artifact_type': 'manifest', 'version': latest_version})['url']

parquet_df = download_parquet_to_df(parquet_url)
manifest = requests.get(manifest_url, timeout=60).json()

# Keep an untouched copy for alternate enrichment flows
parquet_df_original = parquet_df.copy()
print(f"Latest version:{latest_version}")
parquet_df

### Artifact manifest
Quick recap of the downloaded artifact (version, schema, stats).


In [ ]:
display(Markdown(f"**Manifest version:** `{latest_version}`"))
manifest_keys = list(manifest.keys()) if isinstance(manifest, dict) else []
if manifest:
    display(Markdown(f"**Manifest keys:** {manifest_keys}"))
    schema = manifest.get('schema') or manifest.get('columns') or []
    if schema:
        display(Markdown(f"**Schema fields:** {len(schema)}"))
    stats = manifest.get('stats') or {}
    if stats:
        display(Markdown(f"**Stats:** rows={manifest.get('row_count', 'n/a')}, bbox={stats.get('bbox', 'n/a')}"))

## 6. Preview spatial units on a map

Give viewers a visual sanity check before enriching the dataset. The helper expects a geometry column in EPSG:4326.

---


In [ ]:
# Lightweight map preview (centroids + clustering) to avoid browser lag
preview_geometries_map(
    df=parquet_df,
    geometry_column="geometry",
    # render_mode="centroid",        # draw centroids instead of full polygons
    cluster_points=True,  # cluster points when zoomed out
    simplify_tolerance=0.001,  # simplify geometries (~0.001°) if render_mode='geometry'
    max_features=300,  # cap number of features rendered
    popup_columns=["spatial_unit_id"],  # minimal popup to stay light
    tooltip_columns=[],  # disable tooltips (lighter)
    add_layer_control=False,  # skip layer control to reduce DOM
)

In [ ]:
# Reporter for stats + optional map + HTML report (with manifest context)
reporter = AnalyticsRunReporter(
    enabled=True,
    html_path="tmp/analytics_run_report.html",
    include_map=True,
    map_geometry_column="geometry",
    map_max_features=10000,
    map_simplify_tolerance=0.0005,
)
reporter.set_run_context(
    group_id=group_id,
    parquet_version=latest_version,
    manifest=manifest,
    parquet_path=None,
)
reporter.capture_dataset(parquet_df, sample_rows=50, use_for_map=True)


## 7. Enrich with crop mask information

Two approaches:
- **7.a** direct filtering with `target_crop` in the task parameters.
- **7.b** fetch all crops first, then filter locally to get the same result.


**Crop mask parameters**
Use `CropMaskParams` to build the config (same fields as the task schema). Defaults are pre-filled; override what you need.
Use `CropMaskParams` (lib.analytics) to build the dict passed to `run_task`.


In [ ]:
HTML(format_bindings_catalog_html(binding_key=BindingKey.CROP_MASK))

In [ ]:
target_crop_code = "CORN"  # adjust per use case

crop_mask_params_direct = CropMaskParams(
    start_year=2018,
    end_year=2024,
    target_crop=target_crop_code,
    max_workers=20,
    checkpoint_path='tmp/crop_mask_direct_checkpoint.parquet',
    checkpoint_frequency=500,
).model_dump(exclude_none=False)

parquet_df_direct, crop_stats_direct = run_task_with_reporting(
    parquet_df_original.copy(),
    binding_key=BindingKey.CROP_MASK,
    task_params=crop_mask_params_direct,
    field_mapping_override={"geometry": "geometry"},
    progress=True,
    progress_desc="Crop history (direct target)",
    error_csv_path="tmp/errors/crop_mask_errors.csv",
    reporter=reporter,
)

# Default dataset for subsequent steps (publishing, etc.)
parquet_df = parquet_df_direct
parquet_df


### 7.c Optional: fetch all crops
Set `target_crop=None` to retrieve every crop/year; keep `parquet_df_all` as an alternative without manual filtering steps.


In [ ]:
crop_mask_params_all = CropMaskParams(
    start_year=2018,
    end_year=2024,
    target_crop=None,
    max_workers=20,
    checkpoint_path='tmp/crop_mask_all_checkpoint.parquet',
    checkpoint_frequency=50,
).model_dump(exclude_none=False)

parquet_df_all, crop_stats_all = run_task_with_reporting(
    parquet_df_original.copy(),
    binding_key=BindingKey.CROP_MASK,
    task_params=crop_mask_params_all,
    field_mapping_override={"geometry": "geometry"},
    progress=True,
    progress_desc="Crop history (all crops)",
    error_csv_path="tmp/errors/crop_mask_errors.csv",
    reporter=reporter,
)
# To publish the all-crops variant instead of the direct one, reassign:
# parquet_df = parquet_df_all


### Optional: manual filtering from all-crops
After fetching all crops with `target_crop=None`, filter locally to isolate the target crop before publishing.

In [ ]:
manual_filtered = parquet_df_all[
    (parquet_df_all["current_crop_year"] == crop_mask_params_all["end_year"])
    & (parquet_df_all["current_crop"].fillna("").str.upper() == target_crop_code.upper())
].copy()

manual_filtered = filter_crop_history_columns(
    manual_filtered,
    target_crop=target_crop_code,
    history_column="crop_history",
    years_column="crop_history_years",
    output_history_column="crop_history",
    output_years_column="crop_history_years",
    inplace=True,
)
parquet_df_manual = manual_filtered
parquet_df_manual


**Tip:** pass `error_csv_path=\"tmp/errors/crop_mask_errors.csv\"` to `run_task` to export rows with error columns when failures occur. The file is overwritten each run (directories auto-created).

### Choose which version to publish
By default `parquet_df` uses the direct target-crop variant (7.a). To publish the manually filtered alternative, reassign it below.

In [ ]:
# Uncomment to publish the manual filtering variant instead of the direct one.
# parquet_df = parquet_df_manual


### Normalize crop_history_years
Convert the dash-separated string (e.g., `2020-2022-2024`) into a list of ints so downstream analytics can reuse it.

In [ ]:
def to_year_list(val):
    if hasattr(pd, "isna") and pd.isna(val):
        return []
    if isinstance(val, str):
        return [int(x) for x in val.split("-") if x.strip().isdigit()]
    if isinstance(val, (list, tuple)):
        return [int(x) for x in val if pd.notna(x) and str(x).isdigit()]
    try:
        return [int(val)]
    except Exception:
        return []

parquet_df = parquet_df.copy()
if "crop_history_years" in parquet_df.columns:
    parquet_df["crop_history_years_list"] = parquet_df["crop_history_years"].apply(to_year_list)


## 8. Publish the enriched baseline

Push the enriched parquet back to the platform and keep a local copy for reuse.

| Parameter | Description |
| --- | --- |
| `group_id` | Use the group you're enriching |
| `baseline_path` | Local parquet written by this notebook |
| `local_export_dir` | Optional folder to store a copy for offline sharing |
| `description` | Freeform text stored in the artifact manifest |

---


### Execution report

In [ ]:
# Render execution report and auto description
report_html = render_final_report(reporter, show_inline=True)
summary_md = reporter.render_summary_markdown()


### Set artifact description
Be explicit about the description before publishing the artifact (shows up in the catalog).
Update the value in the next cell; leave empty only for scratch runs.


In [ ]:
# Manual description (edit as needed)
description = "Demo Test - CROP MASK + filtered on CORN "

# Or reuse the auto summary from the reporter
description = summary_md


In [ ]:
filename = 'data.parquet'

local_export_dir = Path('local_exports')
local_export_dir.mkdir(exist_ok=True)
local_baseline_path = local_export_dir / f"{group_id}_enriched_baseline.parquet"

confirmation = publish_group_artifact(
    session,
    group_code=group_id,
    df=parquet_df,
    filename=filename,
    persist_path=local_baseline_path,
    description=description,
)

job_status = confirmation.get('job_status', {})
state = job_status.get('state', 'unknown')
message = job_status.get('message', '')

display(Markdown('---'))
display(Markdown('### Artifact publish result'))
display(Markdown(f'- **State:** `{state}`'))
if message:
    display(Markdown(f'- **Message:** {message}'))
display(Markdown(f'- **Local file:** `{local_baseline_path}`'))
display(Markdown('---'))